# 部署至線上端點

若要從應用程式取用模型，您可以將模型部署到線上端點。 您將從本機檔案建立 MLflow 模型，並測試端點。

## 開始之前

您將需要最新版的  **azureml-ai-ml** 套件，才能在此筆記本中執行程式碼。 執行下列資料格以確認已安裝它。

> **注意**：
> 如果未安裝 **azure-ai-ml** 套件，請執行 `pip install azure-ai-ml` 以安裝它。

In [ ]:
## 連線到您的工作區

安裝必要的 SDK 套件之後，現在您已準備好連線到您的工作區。

若要連線到工作區，我們需要識別碼參數 - 訂用帳戶識別碼、資源組名和工作區名稱。 資源組名和工作區名稱已為您填入。 您只需要訂用帳戶識別碼才能完成命令。

若要尋找必要的參數，請按一下 Studio 右上方的訂用帳戶和工作區名稱。 窗格會在右側開啟。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 複製訂用帳戶識別碼，並將 **YOUR-SUBSCRIPTION-ID** 取代為您複製的值。 </p>

## 定義和建立端點

最後，目標是將模型部署至端點。 因此，您必須先建立端點。 端點將是一個 HTTPS 端點，應用程式可以呼叫以從模型接收預測。 應用程式可以使用其 URI 來取用端點，並使用金鑰或權杖進行驗證。

執行下列資料格來定義端點。 請注意，端點的名稱必須是唯一的。 您將使用 函 `datetime` 式來產生唯一的名稱。

In [ ]:
接下來，您將執行下列儲存格來建立端點。 這可能需要數分鐘的時間。 建立端點時，您可以閱讀 [什麼是 Azure Machine Learning 端點](https://learn.microsoft.com/azure/machine-learning/concept-endpoints)。

In [ ]:
<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要！ 等到端點成功建立後再繼續！ 綠色通知應該會出現在 Studio 中。 </p>

## 設定部署

您可以將多個模型部署到端點。 當您想要更新已部署的模型，同時將目前的模型保留在生產環境中時，這最有用。 您必須設定部署，以指定要部署至端點的模型。 在下列資料格中，您將參考模型已定型並儲存在 `model` 本機資料夾中， (儲存在此筆記本的相同資料夾中) 。 請注意，由於您使用 MLflow 模型，因此不需要指定環境或評分腳本。

您也會指定部署模型所需的基礎結構。

## 建立部署

最後，您可以執行下列資料格，實際將模型部署到端點：

In [ ]:
模型的部署可能需要 10-15 分鐘的時間。 等候部署模型時，您可以在 [這段影片中深入瞭解受控端點](https://www.youtube.com/watch?v=SxFGw_OBxNM&ab_channel=MicrosoftDeveloper)。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要！ 等到部署完成，再繼續！ 綠色通知應該會出現在 Studio 中。</p>

因為您只有一個模型部署至端點，所以您希望此部署需要 100% 的流量。 如果您將多個模型部署到端點，您可以使用相同的方法來將流量分散到已部署的模型。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要！ 等到藍色部署設定好再繼續！ 綠色通知應該會出現在 Studio 中。 </p> 

## 測試部署

讓我們藉由叫用端點來測試已部署的模型。 具有範例資料的 JSON 檔案會作為輸入使用。 定型的模型會根據年齡、BMI 和疾病數目等醫療資料，預測患者是否有糖尿病。 表示 `[0]` 病患沒有糖尿病。 表示 `[1]` 病患有糖尿病。

In [ ]:
您可以選擇性地變更檔案中的 `sample-data.json` 值，以嘗試並取得不同的預測。

## 列出端點

雖然您可以在 Studio 中檢視所有端點，但您也可以使用 SDK 列出所有端點：

In [ ]:
## 取得端點詳細資料

如果您想要特定端點的詳細資訊，您也可以使用 SDK 來探索詳細資料。

## 刪除端點和部署

端點一律可供使用，因此無法暫停以節省成本。 若要避免不必要的成本，請刪除端點。

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

The deployment of the model may take 10-15 minutes. While waiting for the model to be deployed, you can learn more about [managed endpoints in this video](https://www.youtube.com/watch?v=SxFGw_OBxNM&ab_channel=MicrosoftDeveloper).

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio.</p>

Since you only have one model deployed to the endpoint, you want this deployment to take 100% of the traffic. If you deploy multiple models to the endpoint, you could use the same approach to distribute traffic across the deployed models.

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the blue deployment is configured before continuing! A green notification should appear in the studio. </p> 

## Test the deployment

Let's test the deployed model by invoking the endpoint. A JSON file with sample data is used as input. The trained model predicts whether a patient has diabetes or not, based on medical data like age, BMI, and the number of pregnancies. A `[0]` indicates a patient doesn't have diabetes. A `[1]` means a patient does have diabetes.

In [ ]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json",
)

if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")

Optionally, you can change the values in the `sample-data.json` file to try and get a different prediction.

## List endpoints

Although you can view all endpoints in the Studio, you can also list all endpoints using the SDK:

In [ ]:
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

## Get endpoint details

If you want more information about a specific endpoint, you can explore the details using the SDK too.

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

## Delete the endpoint and deployment

As an endpoint is always available, it can't be paused to save costs. To avoid unnecessary costs, delete the endpoint.

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)